## Import Section

In [67]:
import pandas as pd
import time
from datetime import datetime
from datetime import date
import os
import glob

## Config Section

In [69]:
path = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting'
ruta_general = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting'
dataframes = []

path_udemy = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/Udemy_msc/User_activity/'
path_netzun = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/Netzun/report_full_students/'

In [70]:
today = date.today()
d1 = today.strftime("%m-%d-%Y")

## Functions Section

In [71]:
def get_concatenated_files_msc(path, folder_name = 'Learner_overview', file_name= 'learner-overview', file_type = '.csv'):
    # path = Contains the path: "C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting"
    # folder_name = Name of the folder that contains the files
    # file_name = common part of the file to search
    # file_type = extension of the file: .csv, .xls, .xlsx
    dataframes = []
    for client_folder in os.listdir(path):
        path_cliente = os.path.join(path, client_folder, folder_name)
        if os.path.isdir(path_cliente):
            files = glob.glob(os.path.join(path_cliente, f'*-{file_name}-*{file_type}'))
            if files:
                # ordenar los archivos por la fecha en su nombre
                files.sort(key=lambda x: x.split(f'-{file_name}-')[-1], reverse=True)
                # abrir el archivo más reciente y cargarlo en un dataframe
                df = pd.read_csv(files[0])
                df = df.dropna(axis=0, how='all')
                df = df.dropna(axis=1, how='all')
                df['Cliente'] = client_folder
                # extract the date from the filename
                date_str = files[0].split(f'-{file_name}-')[1].split('-')[0:3]
                date_str = '-'.join(date_str)
                date = datetime.strptime(date_str, "%Y-%m-%d")
                df['fecha_archivo'] = date.date()
                # agregar el dataframe a la lista
                dataframes.append(df)
    df_final = pd.concat(dataframes)
    return df_final

In [72]:
def get_last_file(path, file_name, file_type, date_format = "%Y-%m-%d"):
    # Buscar todos los archivos que coinciden con el patrón en el directorio
    files = glob.glob(os.path.join(path, f'{file_name}_*{file_type}'))
    
    if not files:
        return None     # No se encontraron archivos
    
    # Ordenar los archivos por fecha y seleccionar el último
    files.sort(key=lambda x: datetime.strptime(x.split(f'{file_name}_')[-1].split('.')[0], date_format), reverse=True)
    
    # Lee el archivo en un pandas DataFrame
    if file_type == '.csv':
        df = pd.read_csv(files[0])
    elif file_type in ['.xlsx', '.xls']:
        df = pd.read_excel(files[0])
    else:
        print('File type not supported')
        return None

    return df  # Devuelve el DataFrame
    
    


## Get Dataframes

In [ ]:
# df_usuarios = get_concatenated_files_msc(path)
# df_usuarios.head(5)

In [73]:
df_progreso = get_concatenated_files_msc(path = path, folder_name = 'Learner_engagement', file_name= 'learner-engagement')
df_progreso.head(3)

,ID Learner,External ID Learner,Last Name,First Name,Email Address,Job Title,Department,City,Manager,Account type,...,Progress (%),Learning Time (min),Score (%),Enrolment Date,Start Date,Completion Date,Cliente,fecha_archivo,Certified,Certificate URL
0,243406.0,NaN,Ochoa Rodríguez,Juan Gabriel,juan.ochoa@griky.co,NaN,Griky,NaN,NaN,user,...,0.0,0.0,0.0,2023-05-29 15:11:10,2023-05-29 15:11:39,NaN,Alata,2023-09-22,NaN,NaN
1,243406.0,NaN,Ochoa Rodríguez,Juan Gabriel,juan.ochoa@griky.co,NaN,Griky,NaN,NaN,user,...,2.0,0.0,0.0,2023-05-30 22:43:01,2023-05-30 22:43:42,NaN,Alata,2023-09-22,NaN,NaN
2,243406.0,NaN,Ochoa Rodríguez,Juan Gabriel,juan.ochoa@griky.co,NaN,Griky,NaN,NaN,user,...,0.0,0.0,NaN,2023-04-18 15:10:24,2023-04-18 15:11:42,NaN,Alata,2023-09-22,NaN,NaN


In [74]:
path_udemy = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/Udemy_msc/User_activity/'
df_udemy = get_last_file(path_udemy, file_name= 'user_activity_report_griky-myskillcamp', file_type='.csv')
df_udemy.head(3)

,User First Name,User Last Name,User Email,External ID,User Groups,User Role,User Is Deactivated,Date Joined,Date of Last Activity,No. of Days Active,No. of New Courses Enrolled,No. of New Courses Started,No. of Courses Completed,Minutes Video Consumed,No. of Lectures Completed,No. of Assigned Courses,No. of Practice Activities Completed
0,Rafael,Rivas,rafael.rivas@griky.co,NaN,NaN,owner,No,2022-11-04,2023-02-24,1,1,1,0,0.25,0,0,0
1,Javier,Pinzón,javier.pinzon@griky.co,NaN,NaN,admin,No,2022-12-07,2023-09-18,37,56,29,1,331.00,77,0,2
2,Anonymized User,Anonymized User,Anonymized User,Anonymized User,NaN,student,Yes,'-,2023-02-06,1,1,1,0,0.25,0,0,0


In [76]:
path_netzun = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/Netzun/report_full_students/'
df_netzun = get_last_file(path_netzun, file_name= 'report_full_students', file_type='.xlsx', date_format= "%m-%d-%Y")
df_netzun.head(3)

,ID,Nombres y Apellidos,Email,Función,Curso,Inscripción del curso,Estado de curso,Lecciones Tomadas/Total Lecciones,Porcentaje de Progreso,Tiempo total,Puntaje Porcentaje,Fecha última lección,Fecha Deadline,Áreas,Código Interno,Filtro
0,288861,Camilo Gil Hernández,camilo.gil155@gmail.com,Usuario,Analítica Digital,08/08/2020,No Iniciado,0/13,0%,0:00:00,-,-,-,EANX,NaN,NaN
1,288861,Camilo Gil Hernández,camilo.gil155@gmail.com,Usuario,Construcción de Lovemarks a través de Insights,09/08/2020,No Iniciado,0/31,0%,0:00:00,-,-,-,EANX,NaN,NaN
2,288861,Camilo Gil Hernández,camilo.gil155@gmail.com,Usuario,Marketing Personal para encontrar tu 1er trabajo,10/08/2020,No Iniciado,0/29,0%,0:00:00,-,-,-,EANX,NaN,NaN


## Save 

In [ ]:
#df_usuarios.to_excel('usuarios.xlsx', index=False)
df_progreso.to_excel(f'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Consolidado_Learner_Engagement/Progreso_MSC_{d1}.xlsx', index=False)